In [1]:
# run w aln output in both modes
import sys
sys.path.append('../src/')
#matcAFDB_tools up positions
#calc fident with cross alphabet comparison

In [2]:
import subprocess ,shlex
def runargs(args):
    '''run a command line command

    Parameters
    ----------
    args : str
        command line command
    '''

    args = shlex.split( args)
    p = subprocess.run( args )
    return p

def runFoldseek_allvall( structfolder , outfolder , foldseekpath = '../foldseek/foldseek' , mode = 2):
    '''
    run foldseek search and createtsv

    parameters
    ----------
    dbpath : str
        path to foldseek database
    outfolder : str 
        path to output folder
    maxseqs : int   
        maximum number of sequences to compare to

    '''

    args = foldseekpath + " easy-search " + structfolder + " "+ structfolder +" "+ outfolder+"aln_"+str(mode)+"_allvall.csv tmp" + " --format-output 'query,target,fident,alnlen,mismatch,gapopen,qstart,qend,tstart,tend,evalue,bits,lddt,qaln,taln,cigar,alntmscore' --exhaustive-search --alignment-type "+ str(mode) 
    p = runargs(args)
    return outfolder+"aln_"+str(mode)+"_allvall.csv"

def read_dbfiles3di(folder, name = 'outdb'):
    #find positions 
    threeDidb = folder+ name +'_ss'
    threeDiseq = [ l.strip().replace('\x00','') for l in open(threeDidb)]
    
    
    lookup = folder+name+'.lookup'
    ids = [ l.split()[1].strip() for l in open(lookup)]
    
    AADB = folder+name
    AAs = [ l.strip().replace('\x00','') for l in open(AADB)]

    mapper3di = dict(zip(ids,threeDiseq))
    mapperAA = dict(zip(ids,AAs))
    
    return mapper3di, mapperAA

In [3]:
mapper3di, mapperAA = read_dbfiles3di('../testdata/')

In [4]:
structfolder = '../testdata/structs'
runFoldseek_allvall(structfolder , outfolder = structfolder, mode = 0)
runFoldseek_allvall(structfolder , outfolder = structfolder, mode = 2)

../testdata/structsaln_0_allvall.csv exists and will be overwritten
easy-search ../testdata/structs ../testdata/structs ../testdata/structsaln_0_allvall.csv tmp --format-output query,target,fident,alnlen,mismatch,gapopen,qstart,qend,tstart,tend,evalue,bits,lddt,qaln,taln,cigar,alntmscore --exhaustive-search --alignment-type 0 

MMseqs Version:              	30fdcac78217579fa25d59bc271bd4f3767d3ebb
Seq. id. threshold           	0
Coverage threshold           	0
Coverage mode                	0
Max reject                   	2147483647
Max accept                   	2147483647
Add backtrace                	false
TMscore threshold            	0
TMalign hit order            	0
TMalign fast                 	1
Preload mode                 	0
Threads                      	48
Verbosity                    	3
LDDT threshold               	0
Sort by structure bit score  	1
Substitution matrix          	aa:3di.out,nucl:3di.out
Alignment mode               	3
Alignment mode               	0
E-value th

'../testdata/structsaln_2_allvall.csv'

In [5]:
#make a foldtree with distmat
import foldseek2tree
import pandas as pd
import numpy as np
outfolder = '../testdata/'
res = pd.read_table('../testdata/structsaln_2_allvall.csv', header = None)
columns = 'query,target,fident,alnlen,mismatch,gapopen,qstart,qend,tstart,tend,evalue,bits,lddt,qaln,taln,cigar,alntmscore'.split(',')
res.columns= columns
ids = list( set(list(res['query'].unique()) + list(res['target'].unique())))
pos = { protid : i for i,protid in enumerate(ids)}
kernels = ['fident', 'alntmscore', 'lddt']
matrices = { k:np.zeros((len(pos), len(pos))) for k in kernels }
print(res)

#calc kernel for tm, aln score, lddt
for idx,row in res.iterrows():
    for k in matrices:
        matrices[k][pos[row['query']] , pos[row['target']]] += row[k]
        matrices[k][pos[row['target']] , pos[row['query']]] += row[k]

trees = {}
for i,k in enumerate(matrices):
    matrices[k] /= 2
    matrices[k] = 1-matrices[k]
    print(matrices[k], np.amax(matrices[k]), np.amin(matrices[k]) )
    np.save( outfolder + k + '_distmat.npy' , matrices[k])
    distmat_txt = foldseek2tree.distmat_to_txt( ids , matrices[k] , outfolder + k + '_distmat.txt' )
    out_tree = foldseek2tree.runFastme(  fastmepath = 'fastme' , clusterfile = distmat_txt )
    out_tree = foldseek2tree.postprocess(out_tree, outfolder + k + 'structblob_tree.nwk' , delta = 10**-10)
    out_tree = foldseek2tree.runargs('../madroot/mad ' + out_tree )
    trees[k] = out_tree


          query      target  fident  alnlen  mismatch  gapopen  qstart  qend  \
0    Q06481.pdb  Q06481.pdb   1.000     763         0        0       1   763   
1    Q06481.pdb  P15943.pdb   0.926     767        51        2       1   763   
2    Q06481.pdb  Q06335.pdb   0.853     732        52        1      32   763   
3    Q06481.pdb  Q95241.pdb   0.499     779       334       12      12   763   
4    Q06481.pdb  P08592.pdb   0.490     789       345       11      12   763   
..          ...         ...     ...     ...       ...      ...     ...   ...   
504  Q95241.pdb  Q28757.pdb   1.000      57         0        0     648   704   
505  Q95241.pdb  Q28053.pdb   1.000      58         0        0     647   704   
506  Q95241.pdb  Q28748.pdb   1.000      57         0        0     648   704   
507  Q95241.pdb  Q28280.pdb   1.000      57         0        0     647   703   
508  Q95241.pdb  Q29149.pdb   1.000      56         0        0     648   703   

     tstart  tend        evalue  bits  

In [6]:
alndf0 = pd.read_table('../testdata/structsaln_0_allvall.csv' , header = None)
alndf2 = pd.read_table('../testdata/structsaln_2_allvall.csv', header = None)
columns = 'query,target,fident,alnlen,mismatch,gapopen,qstart,qend,tstart,tend,evalue,bits,lddt,qaln,taln,cigar,lntmscore'.split(',')
alndf0.columns = columns
alndf2.columns = columns

alndf0['3diq']= alndf0['query'].map(mapper3di)
alndf0['3dit']= alndf0['target'].map(mapper3di)


alndf0['AAq']= alndf0['query'].map(mapperAA)
alndf0['AAt']= alndf0['target'].map(mapperAA)


alndf0 = alndf0.set_index(['query', 'target'])
alndf2 = alndf2.set_index(['query', 'target'])

alndf = alndf0.merge(alndf2, left_index=True, right_on=['query', 'target'])


In [7]:
#use the qaln and taln of x and y to derive the other side fident
import numpy as np

def Fident(str1,str2 , verbose = False):
    #minlen= min( (len(str1),len(str2))  )
    #str1 = str1[:minlen]
    #str2 = str2[:minlen]
    str1 = np.array(list(str1))
    str2 = np.array(list(str2))            
    return len(np.where( (str1 == str2 ) & (str1 != '-' ) & (str2 != '-')  )[0]) / len(str1)

def copyaln( aln, seq):
    seqiter = iter(seq)
    newaln = ''
    for i,char in enumerate(aln):
        if char == '-':
            newaln += '-'
        else:
            newaln+=next(seqiter)
    return newaln

def calc_fident_crossaln(row , verbose = False):
    #amino acid representations of alns using AAand3di or just 3di
    qaln_0,qaln_2, taln_0, taln_2 = row.qaln_x, row.qaln_y, row.taln_x, row.taln_y
    #start and stop of aln
    
    qstart_0,qend_0, tstart_0 , tend_0, qstart_2, qend_2, tstart_2 , tend_2 = row.qstart_x, row.qend_x , row.tstart_x , row.tend_x , row.qstart_y , row.qend_y , row.tstart_y , row.tend_y
    #indexing starts at 1...
    
    #3di of the query and target
    structQ, structT = row['3diq'], row['3dit']
    AAq, AAt = row['AAq'], row['AAt']

    #add gaps
    q3di_newgaps = copyaln(qaln_0, structQ[qstart_0-1:qend_0])
    t3di_newgaps = copyaln(taln_0, structT[tstart_0-1:tend_0])
    t3diAA_newgaps = copyaln(taln_2, structT[tstart_2-1:tend_2]) 
    q3diAA_newgaps = copyaln(qaln_2, structQ[qstart_2-1:qend_2])
    row = pd.Series({ 'fident_3di_mode2' :Fident(q3di_newgaps , t3di_newgaps , verbose = verbose) ,  'fident_3di_mode0' :Fident(q3di_newgaps , t3di_newgaps, verbose = verbose) , 
     '3di_qaln_mode0':q3di_newgaps , '3di_taln_mode0':t3di_newgaps , '3di_qaln_mode2':q3diAA_newgaps , '3di_taln_mode2':t3diAA_newgaps })
    #return columns
    return row

In [8]:
testrow = alndf.iloc[121]
print(testrow)
print(testrow.qaln_y)
print(testrow.qaln_x)
calc_fident_crossaln(testrow, verbose = True)
res = alndf.apply(calc_fident_crossaln , axis = 1)
alndf = pd.concat([alndf,res] , axis = 1)
print(alndf)

fident_x                                                   0.836
alnlen_x                                                     773
mismatch_x                                                   121
gapopen_x                                                      3
qstart_x                                                       1
qend_x                                                       770
tstart_x                                                       1
tend_x                                                       770
evalue_x                                                     0.0
bits_x                                                       813
lddt_x                                                    0.8428
qaln_x         MLPGLALLLLAAWTARALEVPTDGNAGLLAEPQIAMFCGRLNMHMN...
taln_x         MLPGLALLLLAAWTARALEVPTDGNAGLLAEPQIAMFCGRLNMHMN...
cigar_x                                      603M2D84M3I52M1D28M
lntmscore_x                                                0.342
3diq           DDDDDDDDDD

In [9]:
alndf.reset_index(inplace=True)
print(alndf)

          query      target  fident_x  alnlen_x  mismatch_x  gapopen_x  \
0    Q5IS80.pdb  Q5IS80.pdb     1.000       770           0          0   
1    Q5IS80.pdb  P53601.pdb     0.910       770          68          1   
2    Q5IS80.pdb  Q60495.pdb     0.915       772          61          3   
3    Q5IS80.pdb  P79307.pdb     0.863       770         105          0   
4    Q5IS80.pdb  P05067.pdb     0.836       773         121          3   
..          ...         ...       ...       ...         ...        ...   
504  Q29149.pdb  P05067.pdb     0.846        13           2          0   
505  Q29149.pdb  Q06481.pdb     0.785        14           3          0   
506  Q29149.pdb  P12023.pdb     0.846        13           2          0   
507  Q29149.pdb  Q28053.pdb     1.000        10           0          0   
508  Q29149.pdb  P08592.pdb     0.687        16           5          0   

     qstart_x  qend_x  tstart_x  tend_x  ...  \
0           1     770         1     770  ...   
1           2  

In [10]:
#write a small nexus file
with open('template.nx', 'w') as nexout:
    nxstr= """
    #nexus
    begin sets;
        charset part1 = aln1.fasta: *;
        charset part2 = aln2.fasta: *;
        charpartition mine = LG+I+G:part1, 3di_substmat.txt+I+F:part2;
    end;
    """
    nexout.write(nxstr)

In [11]:
#alnscratch 
import os
alnfolder = 'alnscratch/'
os.mkdir('alnscratch')

FileExistsError: [Errno 17] File exists: 'alnscratch'

In [15]:
import toytree
guide_tree = '../testdata/fidentstructblob_tree.nwk.rooted'
#madroot the tree
tre = toytree.tree(guide_tree)
print(tre)



      /-P14599.pdb
   /-|
  |  |   /-Q10651.pdb
  |   \-|
  |      \-Q06BR2.pdb
--|
  |      /-Q03157.pdb
  |   /-|
  |  |   \-P51693.pdb
  |  |
   \-|      /-Q06335.pdb
     |   /-|
     |  |  |   /-Q06481.pdb
     |  |   \-|
      \-|      \-P15943.pdb
        |
        |   /-O93279.pdb
        |  |
         \-|   /-O73683.pdb
           |  |
            \-|   /-Q29149.pdb
              |  |
               \-|   /-Q28748.pdb
                 |  |
                 |  |      /-Q95241.pdb
                  \-|   /-|
                    |  |   \-Q28053.pdb
                    |  |
                     \-|   /-Q28757.pdb
                       |  |
                       |  |      /-Q28280.pdb
                        \-|   /-|
                          |  |  |   /-P53601.pdb
                          |  |   \-|
                          |  |     |   /-Q5IS80.pdb
                           \-|      \-|
                             |         \-P05067.pdb
                             |
    

In [74]:
import subprocess
import os
import glob
import toytree
import tqdm
import pandas as pd
from subprocess import PIPE, Popen
import shlex
from Bio import SeqIO

def get_leafset( treenode ):
    """
    this function returns the leafset of a node
    """
    if treenode.is_leaf():
        return [treenode.name]
    else:
        return treenode.get_leaf_names()

def mergeAlign( fasta1, fasta2, outfile):
    print(fasta1,fasta2)
    #args =  'clustalo --p1 ' + fasta1 + ' --p2 ' + fasta2 + ' --force -o ' + outfile
    print('merge' , fasta1 , fasta2 )
    
    
    args = 'mafft --add ' + fasta1 + ' ' + fasta2 
    args = shlex.split(args)
    p = Popen( args, stdin=PIPE, stdout=PIPE)
    aln = p.communicate()[0].decode()
    print('mafft aln',aln)
    with open( outfile , 'w') as alnout:
        alnout.write(aln)
    
    return outfile

def sub2fasta( sub, outfile , fastacol1='qaln_y' , fastacol2='taln_y' ):
    with open(outfile, 'w') as f:
        f.write('>' + sub['query'] + '\n')
        f.write(sub[fastacol1] + '\n')
        f.write('>' + sub['target'] + '\n')
        f.write(sub[fastacol2] + '\n')    
    return outfile

def retalns(allvall, leafname,leafset):
    sub = allvall[allvall['query'] == leafname]
    sub = sub[sub['target'].isin(leafset)]
    sub = sub[sub['query'] != sub['target']]
    return sub.iloc[0]

#traverse tree from root to leaves recursively
def traverse_tree_merge( treenode, topleafset, allvall ):
    """
    this function traverses a tree from root to leaves recursively
    it returns a dictionary with the iteratively built alignment
    """
    
    if treenode.is_leaf():  
        print(treenode)
        #if the node is a leaf, then we need to add it to the alignment with one of the pivots in the current leafset
        sub = retalns(allvall, treenode.name , topleafset )
        treenode.aln = sub2fasta(sub, alnfolder + treenode.name + '_inter.fasta')
        treenode.aln3di = sub2fasta(sub, alnfolder + treenode.name + '_inter.3di.fasta' , fastacol1='3di_qaln_mode2' , fastacol2='3di_taln_mode2')
        return treenode.aln, treenode.aln3di
    
    else:
        childalns3di = []
        childalnsAA = []
        treenode.leafset = get_leafset(treenode) 
        for c in treenode.get_children():
            if not c.aln:
                c.aln,c.aln3di = traverse_tree_merge(c , treenode.leafset , allvall)
            childalnsAA.append(c.aln)
            childalns3di.append(c.aln3di)
        treenode.aln = mergeAlign( childalnsAA[0], childalnsAA[1] ,  alnfolder + treenode.name + '_inter.fasta')            
        treenode.aln3di = mergeAlign( childalns3di[0], childalns3di[1] , alnfolder + treenode.name + '_inter.3di.fasta' )
        return treenode.aln, treenode.aln3di

def remove_redundant( alignment ):
    """
    this function removes redundant sequences from an alignment
    """
    aln = SeqIO.parse(alignment, 'fasta')
    seqs = []
    ids = []
    for s in aln:
        if s.id not in ids:
            seqs.append(s)
            ids.append(s.id)
    
    with open(alignment, 'w') as f:
        for s in seqs:
            f.write('>' + s.id + '\n')
            f.write(str(s.seq) + '\n')
    return alignment

#remove all alns except the final merged one
def cleanup( filedir ):
    """
    this function removes all alns except the final merged one
    """
    for f in glob.glob(filedir + '*inter.fasta'):
        os.remove(f)


In [ ]:
#prepare tree attributes
for i,n in enumerate(tre.treenode.traverse()):
    n.aln = None
    n.aln3di = None
    n.leafset = None
    if len(n.name) == 0:
        n.name = 'internal_'+str(i)
finalaln, finalaln3di = traverse_tree_merge( tre.treenode.get_tree_root(), get_leafset(tre.treenode.get_tree_root())     , alndf )

In [75]:
print(finalaln, finalaln3di)

alnscratch/44_inter.fasta alnscratch/44_inter.3di.fasta


In [76]:
#copy the final fastas to final fasta

for fasta in [finalaln, finalaln3di]:
    remove_redundant(fasta)

for fasta,data in {'aa':finalaln, '3di':finalaln3di}.items():
    with open(fasta+'.final.fasta' , 'w') as fastout:
        with open( data ) as fastin:
            fastout.write( fastin.read())


In [77]:
#run iqtree
#make partition
def runIQparitioned( fasta1 = 'aa.final.fasta', fasta2 = '3di.final.fasta', nexus= 'template.nx' ):
    print(fasta1,fasta2)
    args = 'iqtree --spp -mdef template.nx'
    args = shlex.split(args)
    p = Popen( args, stdin=PIPE, stdout=PIPE)
    aln = p.communicate()[0].decode()
    print('mafft aln',aln)
    with open( outfile , 'w') as alnout:
        alnout.write(aln)
    
    return outfile